# Tutorial 4 - Hierarchical Agents
- Video: https://www.youtube.com/watch?v=RkY0Et-JhKI

## Functionalities:
- An agent (referred as meta agent) can also assign other agents (referred as inner agents) as the function call, thereby creating a hierarchical structure
- Whenever the inner agent is called, we will run the full inner agent's process until termination
- Maximum number of steps for each agent (meta agent or inner agent) can be defined in `max_subtasks` (default: 5) variable when initialising agent

## Additional Details:
- The inner agent will take in meta agent's instruction as input
- In addition, the inner agent will also have access to meta agent's task and subtasks completed for global context
- Inner agent's output will directly go into meta agent's subtasks completed dictionary, enabling it to directly update global context

## Design Philosophy:
- Bottom layers can get context of the top layers for more context-specific generation
- Top layers choose bottom layers in a hierarchical fashion so as to limit the number of choices at each level for better performance
- `subtasks_completed` serves as the global context for the meta agents and the inner agents, and can be summarised as needed, e.g. when performing a new task to filter out irrelevant information 

# Setup Guide

## Step 1: Install AgentJo

In [1]:
# !pip install agentjo

## Step 2: Import required functions and setup relevant API keys for your LLM

In [ ]:
# Set up API key and do the necessary imports
from agentjo import *
import os

# this is only if you use OpenAI as your LLM
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

## Step 3: Define your own LLM
- Take in a `system_prompt`, `user_prompt`, and outputs llm response string

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Neutral'

# Define Agents and Meta Agents
- You can also create a Meta Agent that uses other Agents (referred to as Inner Agents) as functions
- Create your Meta agent using `Agent()` (Note: No different from usual process of creating Agents - your Meta Agent is also an Agent)
- Set up an Inner Agent list and assign it to your Meta agent using `assign_agents(agent_list)`

## Example Meta Agent Setup
```python
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.',
                 llm = llm)

# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.', llm = llm),
    Agent('Boss', 'Makes decision on which items to include in menu.', llm = llm),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions.', max_subtasks = 1, llm = llm),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them', llm = llm).assign_functions([dish_price])
    ]

my_agent.assign_agents(agent_list)
```

## Run the Meta Agent
- Let us run the agent and see the interactions between the Meta Agent and Inner Agents to solve the task!
```python
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in menu format.')
```

In [5]:
def random_number_from_string(input_string) -> int:
    ''' Returns a random number between 1 and 10 when given an input string '''
    import hashlib
    import random
    # Hash the input string
    hashed_string = hashlib.sha256(input_string.encode()).hexdigest()
    
    # Convert the hashed string to an integer
    hashed_integer = int(hashed_string, 16)
    
    # Seed the random number generator with the hashed integer
    random.seed(hashed_integer)
    
    # Generate a random number between 1 and 10
    return random.randint(1, 10)

In [6]:
# assign a dish pricing function for economist
# Note: Try to make the output informative that is understandable by the Agent - use free text or json to explain the output so that someone can understand without the input
def dish_price(list_of_dish_names: list) -> dict:
    ''' Takes in list_of_dish_names and outputs price of each dish '''
    if not isinstance(list_of_dish_names, list):
        list_of_food_items = [list_of_dish_names]
    output = {}
    for each in list_of_dish_names:
        output[each] = '$'+str(random_number_from_string(each))
    return output

In [7]:
dish_price(['spaghetti', 'tomato'])

{'spaghetti': '$8', 'tomato': '$1'}

In [8]:
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.',
                 llm = llm)

In [9]:
# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.', llm = llm),
    Agent('Boss', 'Makes decisions on which items to include in Menu', llm = llm),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', 
          max_subtasks = 1, llm = llm),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them', llm = llm).assign_functions([dish_price])
    ]

In [10]:
my_agent.assign_agents(agent_list)

# Approach 1: Run a generic task with the Meta-Agent
- Let us run a general task to create 5 menu items for an Italian restaurant
- You can also specify how Agents interact with one another in the task, otherwise the Meta-Agent will infer what should be done

In [11]:
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.')

Observation: No subtasks have been completed yet for the assigned task of generating 5 menu items based on Italian food choices.
Thoughts: To complete the assigned task, I need to first generate interesting dish names and descriptions for Italian cuisine. After that, I will generate the ingredients for each dish and finally determine the pricing.
Subtask identified: Generate interesting dish names and descriptions for 5 Italian menu items.
Calling function Creative Writer with parameters {'instruction': 'Generate interesting dish names and descriptions for 5 Italian menu items.'}

### Start of Inner Agent: Creative Writer ###
Observation: No subtasks have been completed yet for generating interesting dish names and descriptions for 5 Italian menu items.
Thoughts: To complete the assigned task, I need to generate creative and appealing dish names along with their descriptions that reflect Italian cuisine.
Subtask identified: Use the equipped function to generate interesting dish names a

In [12]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate interesting dish names and descriptions for 5 Italian menu items.")
{'Agent Output': '1. **Trattoria di Sogni**: A whimsical pasta dish that combines hand-rolled fettuccine with a rich, creamy truffle sauce, topped with a sprinkle of aged Parmigiano-Reggiano and a drizzle of white truffle oil. Each bite transports you to a cozy Italian trattoria, where dreams and flavors intertwine.\n\n2. **Sole di Napoli**: A vibrant seafood risotto featuring tender calamari, succulent shrimp, and fresh mussels, all simmered in a saffron-infused broth. Garn

In [13]:
my_agent.thoughts

[{'Observation': 'No subtasks have been completed yet for the assigned task of generating 5 menu items based on Italian food choices.',
  'Thoughts': 'To complete the assigned task, I need to first generate interesting dish names and descriptions for Italian cuisine. After that, I will generate the ingredients for each dish and finally determine the pricing.',
  'Current Subtask': 'Generate interesting dish names and descriptions for 5 Italian menu items.',
  'Equipped Function Name': 'Creative Writer',
  'Equipped Function Inputs': {'instruction': 'Generate interesting dish names and descriptions for 5 Italian menu items.'}},
 {'Observation': 'The dish names and descriptions for 5 Italian menu items have been successfully generated using the Creative Writer function.',
  'Thoughts': 'Next, I need to generate the ingredients for each of the 5 menu items. This can be done using the Chef function, which specializes in generating ingredients based on dish names.',
  'Current Subtask': 'Us

In [14]:
output = my_agent.reply_user()

1. **Trattoria di Sogni**
   - **Description**: A whimsical pasta dish that combines hand-rolled fettuccine with a rich, creamy truffle sauce, topped with a sprinkle of aged Parmigiano-Reggiano and a drizzle of white truffle oil. Each bite transports you to a cozy Italian trattoria, where dreams and flavors intertwine.
   - **Ingredients**: Hand-rolled fettuccine, heavy cream, truffle oil, aged Parmigiano-Reggiano, and fresh black truffles.
   - **Price**: $6

2. **Sole di Napoli**
   - **Description**: A vibrant seafood risotto featuring tender calamari, succulent shrimp, and fresh mussels, all simmered in a saffron-infused broth. Garnished with a zesty lemon gremolata, this dish captures the essence of the sun-kissed shores of Naples.
   - **Ingredients**: Arborio rice, calamari, shrimp, mussels, saffron, fish stock, garlic, onion, white wine, and lemon for gremolata.
   - **Price**: $8

3. **Focaccia Fantasia**
   - **Description**: An artisan focaccia bread baked to golden perfecti

# Approach 2: Step through the process yourself as the Meta Agent
- You can use agents manually using `use_agent(agent_name: str, agent_task: str)` The outputs of the agents will be stored into `subtasks_completed` of the meta agent automatically
- You can also interact with the meta agent using `reply_user(task: str, stateful: bool = False)`. This generates a reply for the agent to the `task` using `subtasks_completed` as context. If `stateful` is true, this will also store the output into `subtasks_completed`
- If `subtasks_completed` is too lengthy, you can run `summarise_subtasks_completed(task: str)` This summarises `subtasks_completed` according to what is relevant to the `task`
- If you want to redo a subtask, you can remove the most recent subtask from `subtasks_completed` using `remove_last_subtask()`

In [15]:
# reset the state of the agent
my_agent.reset()
# currently should have no task and nothing in subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [16]:
# Assign a task for the meta agent - this helps to contextualise what you give each of the inner agents later on
my_agent.assign_task('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.')

In [17]:
# This will run the Agent Creative Writer (with shared subtasks_completed as meta agent)
# Output of task will go directly into subtasks_completed - we view the inner agent as simply an extension of the meta agent
my_agent.use_agent('Creative Writer', 'Generate only 5 Italian dish names and descriptions')

Calling function Creative Writer with parameters {'instruction': 'Generate only 5 Italian dish names and descriptions'}

### Start of Inner Agent: Creative Writer ###
Observation: No subtasks have been completed yet for generating Italian dish names and descriptions.
Thoughts: I need to generate 5 unique Italian dish names along with their descriptions. Since there are no specific guidelines or examples provided, I can use my general knowledge of Italian cuisine to create interesting and appealing names and descriptions.
Subtask identified: Generate 5 Italian dish names and their descriptions using the equipped function.
Getting LLM to perform the following task: Generate 5 Italian dish names and their descriptions using the equipped function.
> 1. **Sogni di Pomodoro**: A vibrant tomato-based risotto that dances with the flavors of fresh basil and a hint of garlic, creating a comforting dish that feels like a warm embrace. 

2. **Frittata di Mare**: A delightful seafood frittata, brim

In [18]:
# subtasks_completed should be updated with the Creative Agent's outputs
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate only 5 Italian dish names and descriptions")
{'Agent Output': '1. **Sogni di Pomodoro**: A vibrant tomato-based risotto that dances with the flavors of fresh basil and a hint of garlic, creating a comforting dish that feels like a warm embrace. \n\n2. **Frittata di Mare**: A delightful seafood frittata, brimming with succulent shrimp and tender calamari, all enveloped in a fluffy egg mixture, perfect for a seaside brunch. \n\n3. **Cappellacci di Ricotta e Spinaci**: Handcrafted pasta pockets filled with creamy ricotta and fresh spinach, serv

In [19]:
my_agent.use_agent('Chef', 'Generate ingredients for the 5 Italian dishes')

Calling function Chef with parameters {'instruction': 'Generate ingredients for the 5 Italian dishes'}

### Start of Inner Agent: Chef ###
Observation: The task of generating Italian dish names and descriptions has been completed, providing a list of five distinct dishes.
Thoughts: Now, I need to generate the ingredients for each of the five Italian dishes listed. Since there are no specific functions available for generating ingredients, I will use the general function to create a response.
Subtask identified: Use the general function to generate a list of ingredients for each of the five Italian dishes: Sogni di Pomodoro, Frittata di Mare, Cappellacci di Ricotta e Spinaci, Pollo al Limone e Rosmarino, and Torta della Nonna.
Getting LLM to perform the following task: Use the general function to generate a list of ingredients for each of the five Italian dishes: Sogni di Pomodoro, Frittata di Mare, Cappellacci di Ricotta e Spinaci, Pollo al Limone e Rosmarino, and Torta della Nonna.
> 

In [20]:
my_agent.use_agent('Economist', 'Generate prices for the 5 Italian dishes')

Calling function Economist with parameters {'instruction': 'Generate prices for the 5 Italian dishes'}

### Start of Inner Agent: Economist ###
Observation: The dish names have been generated along with their descriptions and ingredients, but no pricing has been established yet for the 5 Italian dishes.
Thoughts: To complete the Assigned Task, I need to utilize the dish names provided to generate prices for each dish. Since I have the names, I can directly input them into the appropriate function to obtain the pricing.
Subtask identified: Use the dish names of the 5 Italian dishes to generate their prices.
Calling function dish_price with parameters {'list_of_dish_names': ['Sogni di Pomodoro', 'Frittata di Mare', 'Cappellacci di Ricotta e Spinaci', 'Pollo al Limone e Rosmarino', 'Torta della Nonna']}
> {'output_1': {'Sogni di Pomodoro': '$9', 'Frittata di Mare': '$2', 'Cappellacci di Ricotta e Spinaci': '$3', 'Pollo al Limone e Rosmarino': '$10', 'Torta della Nonna': '$8'}}

Observatio

In [21]:
my_agent.use_agent('Boss', 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay')

Calling function Boss with parameters {'instruction': 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay'}

### Start of Inner Agent: Boss ###
Observation: I have generated five Italian dish names and descriptions, their ingredients, and established prices for each dish. However, no subtasks have been completed yet for selecting three dishes that would please Gordon Ramsay.
Thoughts: To complete the assigned task, I need to analyze the five dishes based on their descriptions, ingredients, and prices to determine which three would be most appealing to Gordon Ramsay, considering his culinary standards and preferences.
Subtask identified: Evaluate the five Italian dishes and select three that would be most likely to impress Gordon Ramsay based on their quality, presentation, and flavor profiles.
Getting LLM to perform the following task: Evaluate the five Italian dishes and select three that would be most likely to impress Gordon Ramsay based on their quality, presentat

In [22]:
# Get a targeted response from your agent, and add this to subtask_completed
output = my_agent.reply_user('Choose the 3 menu items for Gordon Ramsay line by line in this format - Dish Name, Dish Description, Dish Ingredients, Dish Price')

1. **Sogni di Pomodoro**: A vibrant tomato-based risotto that dances with the flavors of fresh basil and a hint of garlic, creating a comforting dish that feels like a warm embrace. 
   *Ingredients*: Arborio rice, fresh tomatoes, vegetable broth, garlic, fresh basil, olive oil, onion, and Parmesan cheese. 
   *Price*: $9  

2. **Cappellacci di Ricotta e Spinaci**: Handcrafted pasta pockets filled with creamy ricotta and fresh spinach, served with a drizzle of sage-infused brown butter, offering a taste of rustic Italian charm. 
   *Ingredients*: Flour (for pasta), ricotta cheese, fresh spinach, egg, sage, butter, and nutmeg. 
   *Price*: $3  

3. **Pollo al Limone e Rosmarino**: Juicy chicken thighs marinated in zesty lemon and fragrant rosemary, roasted to perfection, delivering a burst of Mediterranean sunshine in every bite. 
   *Ingredients*: Chicken thighs, lemon, fresh rosemary, garlic, olive oil, salt, and pepper. 
   *Price*: $10


In [23]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate only 5 Italian dish names and descriptions")
{'Agent Output': '1. **Sogni di Pomodoro**: A vibrant tomato-based risotto that dances with the flavors of fresh basil and a hint of garlic, creating a comforting dish that feels like a warm embrace. \n\n2. **Frittata di Mare**: A delightful seafood frittata, brimming with succulent shrimp and tender calamari, all enveloped in a fluffy egg mixture, perfect for a seaside brunch. \n\n3. **Cappellacci di Ricotta e Spinaci**: Handcrafted pasta pockets filled with creamy ricotta and fresh spinach, serv

In [24]:
# you can summarise the subtasks_history for the next task
my_agent.summarise_subtasks_completed('Generate the 3 Italian dishes for Gordon Ramsay in menu format')

1. **Sogni di Pomodoro**: A vibrant tomato-based risotto that dances with the flavors of fresh basil and a hint of garlic, creating a comforting dish that feels like a warm embrace.  
   *Ingredients*: Arborio rice, fresh tomatoes, vegetable broth, garlic, fresh basil, olive oil, onion, and Parmesan cheese.  
   *Price*: $9  

2. **Cappellacci di Ricotta e Spinaci**: Handcrafted pasta pockets filled with creamy ricotta and fresh spinach, served with a drizzle of sage-infused brown butter, offering a taste of rustic Italian charm.  
   *Ingredients*: Flour (for pasta), ricotta cheese, fresh spinach, egg, sage, butter, and nutmeg.  
   *Price*: $3  

3. **Pollo al Limone e Rosmarino**: Juicy chicken thighs marinated in zesty lemon and fragrant rosemary, roasted to perfection, delivering a burst of Mediterranean sunshine in every bite.  
   *Ingredients*: Chicken thighs, lemon, fresh rosemary, garlic, olive oil, salt, and pepper.  
   *Price*: $10


In [25]:
# subtask has been summarised, helping to reduce the context significantly for later task generation
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Current Results for 'Generate the 3 Italian dishes for Gordon Ramsay in menu format'
1. **Sogni di Pomodoro**: A vibrant tomato-based risotto that dances with the flavors of fresh basil and a hint of garlic, creating a comforting dish that feels like a warm embrace.  
   *Ingredients*: Arborio rice, fresh tomatoes, vegetable broth, garlic, fresh basil, olive oil, onion, and Parmesan cheese.  
   *Price*: $9  

2. **Cappellacci di Ricotta e Spinaci**: Handcrafted pasta pockets filled with creamy ricotta and fresh spinach, served with a drizzle of sage-infused brown butter, offeri

In [26]:
my_agent.subtasks_completed

{"Current Results for 'Generate the 3 Italian dishes for Gordon Ramsay in menu format'": '1. **Sogni di Pomodoro**: A vibrant tomato-based risotto that dances with the flavors of fresh basil and a hint of garlic, creating a comforting dish that feels like a warm embrace.  \n   *Ingredients*: Arborio rice, fresh tomatoes, vegetable broth, garlic, fresh basil, olive oil, onion, and Parmesan cheese.  \n   *Price*: $9  \n\n2. **Cappellacci di Ricotta e Spinaci**: Handcrafted pasta pockets filled with creamy ricotta and fresh spinach, served with a drizzle of sage-infused brown butter, offering a taste of rustic Italian charm.  \n   *Ingredients*: Flour (for pasta), ricotta cheese, fresh spinach, egg, sage, butter, and nutmeg.  \n   *Price*: $3  \n\n3. **Pollo al Limone e Rosmarino**: Juicy chicken thighs marinated in zesty lemon and fragrant rosemary, roasted to perfection, delivering a burst of Mediterranean sunshine in every bite.  \n   *Ingredients*: Chicken thighs, lemon, fresh rosemar

In [27]:
# we also allow you to manually remove the last item added to subtasks_completed, so you can revert your agent state if needed for backtracking
my_agent.remove_last_subtask()

Removed last subtask from subtasks_completed: ("Current Results for 'Generate the 3 Italian dishes for Gordon Ramsay in menu format'", '1. **Sogni di Pomodoro**: A vibrant tomato-based risotto that dances with the flavors of fresh basil and a hint of garlic, creating a comforting dish that feels like a warm embrace.  \n   *Ingredients*: Arborio rice, fresh tomatoes, vegetable broth, garlic, fresh basil, olive oil, onion, and Parmesan cheese.  \n   *Price*: $9  \n\n2. **Cappellacci di Ricotta e Spinaci**: Handcrafted pasta pockets filled with creamy ricotta and fresh spinach, served with a drizzle of sage-infused brown butter, offering a taste of rustic Italian charm.  \n   *Ingredients*: Flour (for pasta), ricotta cheese, fresh spinach, egg, sage, butter, and nutmeg.  \n   *Price*: $3  \n\n3. **Pollo al Limone e Rosmarino**: Juicy chicken thighs marinated in zesty lemon and fragrant rosemary, roasted to perfection, delivering a burst of Mediterranean sunshine in every bite.  \n   *Ingr

In [28]:
# visualise your agent's status - should have empty subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed: None
Is Task Completed: False


In [29]:
my_agent.subtasks_completed

{}

# More Advanced Agents with Shared Variables
- Each Inner Agent has access to a specific part of Shared Variables via Global Context
- Each Inner Agent will update the Shared Variables as it is changed by their actions
- Meta Agent will know of the change in inventory via Shared Variables, as well as the reply of the Inner Agent

In [30]:
def add_item_to_inventory(shared_variables, item: str) -> str:
    ''' Adds item to inventory, and returns outcome of action '''
    shared_variables['Inventory A'].append(item)
    return f'{item} successfully added to Inventory A'

def remove_item_from_inventory(shared_variables, item: str) -> str:
    ''' Removes item from inventory and returns outcome of action '''
    if item in shared_variables['Inventory A']:
        shared_variables['Inventory A'].remove(item)
        return f'{item} successfully removed from Inventory A'
    else:
        return f'{item} not found in Inventory A, unable to remove'

In [31]:
def add_item_to_inventory_2(shared_variables, item: str) -> str:
    ''' Adds item to inventory, and returns outcome of action '''
    shared_variables['Inventory B'].append(item)
    return f'{item} successfully added to Inventory B'
    
def remove_item_from_inventory_2(shared_variables, item: str) -> str:
    ''' Removes item from inventory and returns outcome of action '''
    if item in shared_variables['Inventory B']:
        shared_variables['Inventory B'].remove(item)
        return f'{item} successfully removed from Inventory B'
    else:
        return f'{item} not found in Inventory B, unable to remove'

In [32]:
agent_a = Agent('Factory A Inventory Manager', 
      'Adds and removes items in Inventory A. Only able to remove items if present in Inventory A',
      global_context = 'Inventory of Factory A: <Inventory A>', # Add in Global Context here with shared_variables Inventory
      llm = llm).assign_functions([add_item_to_inventory, remove_item_from_inventory])

agent_b = Agent('Factory B Inventory Manager', 
      'Adds and removes items in Inventory B. Only able to remove items if present in Inventory B',
      global_context = 'Inventory of Factory B: <Inventory B>', # Add in Global Context here with shared_variables Inventory
      llm = llm).assign_functions([add_item_to_inventory_2, remove_item_from_inventory_2])

meta_agent = Agent('Inventory Manager', 'Manages inventory of all factories', 
       shared_variables = {'Inventory A': [],
                           'Inventory B': []},
        llm = llm,
       global_context = 'Inventory of Factory A: <Inventory A>\nInventory of Factory B: <Inventory B>').assign_functions([agent_a, agent_b])

In [33]:
meta_agent.print_functions()

Name: use_llm
Description: For general tasks. Used only when no other function can do the task
Input: []
Output: {'Output': 'Output of LLM'}

Name: end_task
Description: Passes the final output to the user
Input: []
Output: {}

Name: Factory A Inventory Manager
Description: Agent Description: ```Adds and removes items in Inventory A. Only able to remove items if present in Inventory A```
Executes the given <instruction>
Input: ['instruction']
Output: {'Agent Output': 'Output of instruction'}

Name: Factory B Inventory Manager
Description: Agent Description: ```Adds and removes items in Inventory B. Only able to remove items if present in Inventory B```
Executes the given <instruction>
Input: ['instruction']
Output: {'Agent Output': 'Output of instruction'}



In [34]:
meta_agent.run('Add apples to Factory A, add pears to Factory B')

Observation: No items have been added to either Factory A or Factory B yet.
Thoughts: To complete the Assigned Task, I need to add apples to Factory A and pears to Factory B. Since both inventories are currently empty, I can start by adding apples to Factory A first.
Subtask identified: Add apples to Factory A.
Calling function Factory A Inventory Manager with parameters {'instruction': 'Add apples to Factory A.'}

### Start of Inner Agent: Factory A Inventory Manager ###
Observation: No items have been added to Inventory A yet, and the inventory is currently empty.
Thoughts: To complete the assigned task of adding apples to Factory A, I need to use the function that allows me to add items to the inventory.
Subtask identified: Add apples to Inventory A using the add_item_to_inventory function.
Calling function add_item_to_inventory with parameters {'item': 'apples'}
> {'output_1': 'apples successfully added to Inventory A'}

Observation: The task to add apples to Inventory A has alread

[{'Agent Output': 'The assigned task was to add apples to Factory A. However, according to the global context, apples are already present in Inventory A. The subtasks completed indicate that the action to add apples was executed, resulting in the output: "apples successfully added to Inventory A." Therefore, the current state of Inventory A remains unchanged with the presence of apples.'},
 {'Agent Output': 'The assigned task was to execute the instruction to add pears to Factory B. However, according to the global context, pears are already present in Inventory B. The subtasks completed indicate that the action to add pears was executed, resulting in the output: "pears successfully added to Inventory B." Therefore, the current state of Inventory B remains unchanged with the presence of pears.'}]

In [35]:
meta_agent.shared_variables

{'Inventory A': ['apples'],
 'Inventory B': ['pears'],
 'agent': <agentjo.agent.Agent at 0x12aeb26d0>}

In [36]:
meta_agent.run('Remove apples from Factory A, and add bananas and kiwi')

Observation: The subtasks completed indicate that apples were successfully added to Factory A, but this is redundant since apples were already present in Inventory A. Additionally, pears were added to Factory B, which was also unnecessary as they were already in the inventory.
Thoughts: To complete the assigned task, I need to remove apples from Factory A and add bananas and kiwi. Since the removal of apples is the first step, I will focus on that before adding the new items.
Subtask identified: Remove apples from Factory A.
Calling function Factory A Inventory Manager with parameters {'instruction': 'Remove apples from Factory A.'}

### Start of Inner Agent: Factory A Inventory Manager ###
Observation: The assigned task was to remove apples from Factory A. However, the subtasks completed indicate that no actions have been taken yet regarding this task.
Thoughts: Since apples are present in Inventory A, I can proceed to remove them. The next step is to execute the removal of apples fro

[{'Agent Output': 'The assigned task was to add apples to Factory A. However, according to the global context, apples are already present in Inventory A. The subtasks completed indicate that the action to add apples was executed, resulting in the output: "apples successfully added to Inventory A." Therefore, the current state of Inventory A remains unchanged with the presence of apples.'},
 {'Agent Output': 'The assigned task was to execute the instruction to add pears to Factory B. However, according to the global context, pears are already present in Inventory B. The subtasks completed indicate that the action to add pears was executed, resulting in the output: "pears successfully added to Inventory B." Therefore, the current state of Inventory B remains unchanged with the presence of pears.'},
 {'Agent Output': 'The assigned task was to remove apples from Factory A. According to the global context, apples are present in Inventory A. The subtasks completed indicate that the action to

In [37]:
meta_agent.shared_variables

{'Inventory A': ['bananas', 'kiwi'],
 'Inventory B': ['pears'],
 'agent': <agentjo.agent.Agent at 0x12aeb26d0>}

In [38]:
meta_agent.status()

Agent Name: Inventory Manager
Agent Description: Manages inventory of all factories
Available Functions: ['use_llm', 'end_task', 'Factory A Inventory Manager', 'Factory B Inventory Manager']
Shared Variables: ['Inventory A', 'Inventory B', 'agent']
Task: Remove apples from Factory A, and add bananas and kiwi
Subtasks Completed:
Subtask: Factory A Inventory Manager(instruction="Add apples to Factory A.")
{'Agent Output': 'The assigned task was to add apples to Factory A. However, according to the global context, apples are already present in Inventory A. The subtasks completed indicate that the action to add apples was executed, resulting in the output: "apples successfully added to Inventory A." Therefore, the current state of Inventory A remains unchanged with the presence of apples.'}

Subtask: Factory B Inventory Manager(instruction="Execute the instruction to add pears to Factory B.")
{'Agent Output': 'The assigned task was to execute the instruction to add pears to Factory B. Howe